# 미션 3: RAG 기초 이론 & 실습


## 🎯 학습 목표

이 미션에서는 RAG(Retrieval-Augmented Generation)의 기초 개념을 학습하고 간단한 RAG 시스템을 구현합니다.

### 주요 학습 내용
- RAG의 개념과 원리
- 문서 임베딩과 벡터 DB
- 검색 기반 답변 생성
- LangChain을 활용한 RAG 구현


## 📝 실습 내용

### 구현 기능
- 문서 데이터 로드 및 전처리
- 문서 임베딩 생성
- 벡터 DB 구축 (FAISS/ChromaDB 등)
- 질문에 대한 관련 문서 검색
- 검색 결과 기반 답변 생성

### 사용 기술
- Python
- LangChain
- OpenAI Embeddings
- Vector Database (FAISS/ChromaDB)
- Jupyter Notebook


## LLM 모델
- gpt-3.5-turbo

## 라이브러리 로드

In [51]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

import os
from dotenv import load_dotenv
load_dotenv()

True

In [52]:
api_key = os.environ['OPENAI_API_KEY'] 

## 문서 데이터 로드 및 전처리

In [55]:
loader = PyPDFLoader("./data/차세대 한국형 스마트팜 개발.pdf") 
data  = loader.load()

In [58]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len
)

text_splitter.split_documents(pages)

[Document(metadata={'producer': 'Skia/PDF m51', 'creator': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.63 Safari/537.36', 'creationdate': '2018-08-23T06:06:07+00:00', 'moddate': '2018-08-23T06:06:07+00:00', 'source': './data/차세대 한국형 스마트팜 개발.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='차세대\xa0한국형스마트팜\xa0개발\n국립농업과학원\xa0농업공학부\xa0스마트팜개발과\xa0이현동\xa0연구관\n한국형\xa0스마트팜\xa0개발\xa0프로젝트\xa0!\n우리나라\xa0농업은\xa0고령화와\xa0높은\xa0노동강도로\xa0인해\xa0청년인구의\xa0유입이\xa0어려운\xa0구조로\xa0타\xa0분야보다\xa0생산인구\xa0절벽화가\xa0더욱\n가시화되고\xa0있다.\xa0이에\xa0대한\xa0대안으로\xa0제시되고\xa0있는\xa0스마트팜은\xa0사물인터넷,\xa0빅데이터,\xa0인공지능,\xa0로봇\xa0등을\xa0활용하여\n농산물의\xa0생육환경을\xa0최적상태로\xa0관리하고\xa0노동력\xa0절감과\xa0생산성\xa0향상을\xa0구현하는\xa0효율적인\xa0농업형태이다.\xa0\n차세대\xa0한국형\xa0스마트팜\xa0기술개발\xa0프로젝트는\xa04기관\xa019개\xa0전담부서가\xa0협업하여\xa0핵심\xa0요소\xa0및\xa0원천\xa0기반기술의\xa0확보를\xa0위해\n연구\xa0역량을\xa0집중하고\xa0있고\xa0국내\xa0농업여건에\xa0적합하게\xa0기술수준별로\xa0스마트팜\xa0모델을\xa03가지\xa0단계로\xa0구분하여\xa0개발을\n추진하고\xa0있다.\xa0단계별\xa

## 문서 임베딩 생성

In [59]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)

In [60]:
vectorstore = FAISS.from_documents(data, embeddings)

## 질문에 대한 관련 문서 검색

In [61]:
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})


In [72]:
query = "한국형 스마트팜이란?"
result = faiss_retriever.get_relevant_documents(query)

In [73]:
print(result[0].page_content)
print('---------------------------------')
print(result[1].page_content)

차세대 한국형스마트팜 개발
국립농업과학원 농업공학부 스마트팜개발과 이현동 연구관
한국형 스마트팜 개발 프로젝트 !
우리나라 농업은 고령화와 높은 노동강도로 인해 청년인구의 유입이 어려운 구조로 타 분야보다 생산인구 절벽화가 더욱
가시화되고 있다. 이에 대한 대안으로 제시되고 있는 스마트팜은 사물인터넷, 빅데이터, 인공지능, 로봇 등을 활용하여
농산물의 생육환경을 최적상태로 관리하고 노동력 절감과 생산성 향상을 구현하는 효율적인 농업형태이다. 
차세대 한국형 스마트팜 기술개발 프로젝트는 4기관 19개 전담부서가 협업하여 핵심 요소 및 원천 기반기술의 확보를 위해
연구 역량을 집중하고 있고 국내 농업여건에 적합하게 기술수준별로 스마트팜 모델을 3가지 단계로 구분하여 개발을
추진하고 있다. 단계별 스마트팜은 1세대(편리성 증진), 2세대(생산성 향상-네덜란드추격형), 3세대(글로벌산업화-플랜트
수출형)로 구분되고 기술의 단계적 개발과 실용화 계획을 통해 노동력과 농자재의 사용을 줄이고, 생산성과 품질을
제고함으로 농가소득과 연계하며, 나아가 영농현장의 애로와 연관 산업의 문제를 동시에 해결해 간다는 계획이다.
차세대 한국형 스마트팜 융복합 프로젝트
세대별 스마트팜 모델의 특징은 ?
차세대 한국형 스마트팜 프로젝트는 외국의 선진 기술을 그대로 수입・적용하거나 단순 모방하지 않고 우리 농업 환경과
현장 상황에 적합한 기술을 개발하여 농업선진국과 경쟁할 수 있는 독자적인 농업생산기술을 확보하는 것이다.
1세대 스마트팜 - 원격 모니터링과 제어로 편의성 향상
원격 관리를 이용한 농업인 편리성 증대를 목적으로 현재까지 개발한 자동화 및 ICT 기술들을 영농여건과 농가수준에 맞추어 기본형과
선택형으로 구분해 모델을 제시함으로써 농가의 필요에 따라 조건에 맞는 모델을 적용할 수 있도록 한 것이다.
모델유형 주요기능 모델유형 주요기능 모델유형 주요기능
1세대 스마트온실 모델 4종(기본형1, 선택형3)
---------------------------------
2016년 개발 완

## 검색 결과 기반 답변 생성

In [65]:


llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

In [66]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [67]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    memory=memory
)

In [70]:
query = "한국형 스마트팜이란?"
result = conversation_chain({"question": query})
answer = result["answer"]
print(answer)

한국형 스마트팜에서 필요한 ICT 기술은 사물인터넷, 빅데이터, 인공지능, 로봇 등을 활용하는 기술들이 필요합니다. 이러한 기술들을 통해 농산물의 생육환경을 최적화하고 노동력을 절감하며 생산성을 향상시키는 효율적인 농업형태를 구현할 수 있습니다.
